안녕하세요! 데이크루 1기로 활동 중인 sssssun입니다!

계속해서 sklearn을 활용한 머신러닝에 대해 공부해볼텐데요. 

오늘은 **교차검증(Cross Validaiton)**에 대해 포스팅해보겠습니다.

저도 처음 교차검증을 배워보며 공부한 내용을 작성하는 것이기 때문에 다른 팁이나 잘못된 내용이 있다면 꼭 알려주세요😇

### **1. 교차검증을 하는 이유?**

우리가 머신러닝 모델을 만들고 모델의 성능을 높이기 위해 우리는 데이터셋을 **학습용 데이터(train set)**와 **테스트용 데이터(test set)**로 나눕니다.

sklearn의 경우 **train_test_split** 메서드로 과정을 수행했죠.

이 경우에 우리는 결국 test data를 사용하여 모델 검증을 하게 됩니다. 

하지만 고정된 test set을 가지고 모델의 성능을 확인하고 파라미터를 수정하고 이 과정을 반복하면, 결국 우리가 만든 모델은 test set에만 잘 동작하는 모델이 됩니다. 

이렇게 머신러닝에서 test set을 과하게 잘 학습하여 다른 실제 데이터를 가지고 예측을 수행하면 정확성이 떨어지는 것을 **과적합(overfitting)**이라고 합니다.

이러한 과적합의 방지를 위해 교차 검증이 필요합니다!

### **2. 교차 검증이란?**

주어진 데이터셋에 학습된 알고리즘이 얼마나 잘 일반화되어있는지 평가하기 위한 방법입니다.

데이터를 여러 번 반복해서 나누고 여러번의 모델 학습과 검증 과정을 거칩니다.

교차 검증을 활용하면 데이터셋 내의 **모든 데이터를 훈련에 활용**할 수 있고, 모델의 **성능과 정확도를 더 향상**시킬 수 있으며 좀 더 **일반화된 모델**을 만들 수 있습니다.

또한 데이터 부족으로 인한 underfitting을 방지할 수 있고, test set이 편중되는 것을 방지할 수 있습니다.


오늘은 교차검증 방법 중에 가장 대중화된 **K-Fold Cross Validation(k-겹 교차 검증)**과 **Stratified K-Fold Cross Validation(계층별 k-겹 교차 검증)**을 다뤄보겠습니다.

[sklearn으로 파이썬 머신러닝 입문하기🔥 - 분류 모델](https://dacon.io/codeshare/4421) <- 이전 포스팅에서 구현했던 iris data set을 활용한 의사결정나무 분류 모델을 이용하겠습니다!

In [14]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

필요한 라이브러리들을 import 해주었습니다.

In [2]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=1)
dt_clf = DecisionTreeClassifier(random_state =1)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
round(accuracy_score(y_test, pred), 4)

0.9556

의사결정나무 모델을 구현하여 모델의 정확도를 산출해보았습니다.

0.9556이 나오네요.

이제 교차검증을 활용해보겠습니다.

### **3. K-Fold Cross Validation(k-겹 교차 검증)**


k겹 교차 검증에서는 먼저 우리가 아는 것처럼 전체 데이터셋을 **train set** 과 **test set**으로 나눠줍니다.

그리고 train set을 **k개의 폴드 세트(fold set)**으로 만들어주고, k개의 폴드 세트를 **train set(학습용)**과 **validation set(검증용)**으로 나누게 됩니다.

k-1개를 학습 데이터셋으로, 1개를 검증 데이터셋으로 설정하여 검증 평가를 진행하는데, k개의 폴드 세트에 대해 차례로 검증 데이터셋을 변경하여 **k번의 검증**을 수행합니다.

모델을 학습한 뒤, **k번의 성능 평가 결과를 평균**하여 최종 모델 성능을 구합니다.

먼저 sklearn.model_selection에서 **kFold** 메서드를 import 해주었습니다.

In [3]:
from sklearn.model_selection import KFold

**n_splits**는 몇 개의 폴드(fold)로 나눌 것인지를 의미하는 매개변수로 사용자가 임의로 선택할 수 있습니다.

5겹 교차 검증이 가장 일반적이라 저는 5를 넣어 kfold 객체를 생성해주었습니다.

In [4]:
kfold = KFold(n_splits=5)

iris 피처 데이터셋과 레이블 데이터셋을 각각 feature 와 label로 지정해주었습니다.

In [20]:
iris = load_iris()
feature = iris.data
label = iris.target

조건문을 활용하여 폴드 세트 별로 교차 검증의 정확도와 데이터셋의 활용이 어떻게 진행되는지 인덱싱을 나타내보겠습니다.

먼저 폴드 세트 별로 성능 평가 결과를 저장할 리스트 객체 **cv_accuracy**를 만들었고, 검증 횟수를 나타내는 **n_iter** 를 만들어주었습니다.

그리고 의사결정나무 모델의 객체를 생성해주었습니다.

In [6]:
cv_accuracy = []
n_iter =0
dt_clf = DecisionTreeClassifier(random_state=1)

In [26]:
feature.shape

(150, 4)

피처 데이터셋이 150개이니, 5개의 폴드로 나누면 30개씩이겠군요.

인덱스를 출력하여 직접 확인해보겠습니다!

split 함수를 활용하면 kfold의 학습용 데이터 인덱스, 테스트용 데이터 인덱스를 ,array로 반환해줍니다.

그리고 추출한 인덱스를 통해 데이터를 인덱싱하여 모델을 학습, 검증하는 과정을 거칩니다.

마지막으로 폴드 세트 별로 정확도와 데이터 크기, 데이터 인덱스를 출력해보았습니다.

In [40]:
n_iter =0
for train_index, test_index in kfold.split(feature):  
    x_train, x_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 : 1.0,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 1.0,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.9,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]


30개씩 5개의 세트로 나누어 30 단위씩 인덱스가 바뀌며 검증 데이터셋이 바뀌는 것을 볼 수 있습니다.

그리고 5개의 폴드 세트에 대한 교차 검증 정확도를 평균내어 최종 모델 성능을 구해보았습니다.

In [24]:
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


## 평균 검증 정확도: 0.91332


교차검증 이전의 정확도보다는 떨어졌지만, 과적합이 교정된 것으로 이해하면 될 것 같습니다.

### **4. KFold의 문제점**

KFold에는 약간의 문제점이 존재하는데, 먼저 KFold 는 데이터셋을 일정한 간격으로 쪼갭니다.

가령 iris data set을 보면, 레이블이 0,1,2 세개가 존재하고 레이블 0인 데이터셋 50개, 1인 데이터셋 50개, 2인 데이터셋 50개의 순서로 이루어져있죠.

iris data set을 data frame으로 나타내보았습니다.

In [36]:
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


따라서, 분류한 폴드세트에서 학습 데이터셋의 레이블이 0, 1로만 이루어져있다면 데이터셋의 레이블이 2인 데이터는 올바르게 분류해낼 수 없습니다.

In [41]:
kfold_1 = KFold(n_splits=3)
n_iter_=0

for train_index, test_index in kfold_1.split(iris_df):  
    n_iter_+=1
    train = iris_df['label'].iloc[train_index]
    test = iris_df['label'].iloc[test_index]
    
    print('# 교차 검증:{}'.format(n_iter_))
    print('# 학습 레이블 데이터 : \n',train.value_counts())
    print('# 검증 레이블 데이터 : \n',test.value_counts())

# 교차 검증:1
# 학습 레이블 데이터 : 
 1    50
2    50
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    50
Name: label, dtype: int64
# 교차 검증:2
# 학습 레이블 데이터 : 
 0    50
2    50
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 1    50
Name: label, dtype: int64
# 교차 검증:3
# 학습 레이블 데이터 : 
 0    50
1    50
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 2    50
Name: label, dtype: int64


만약 폴드 세트를 3개로 쪼갠다면, 이렇게 극단적인 모습으로 레이블 데이터가 나눠집니다.

In [42]:
kfold = KFold(n_splits=3)
n_iter =0
for train_index, test_index in kfold.split(feature):  
    x_train, x_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 : 0.0,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]

#2 교차 검증 정확도 : 0.0,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]

#3 교차 검증 정확도 : 0.0,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]


그리고... 교차 검증의 정확도 또한 보시다시피 0이 됩니다.

이렇게 KFold는 데이터 편향이 존재하는 경우, 제대로된 모델 학습과 검증이 이루어지지 않을 수 있습니다.

이를 해결할 수 있는 방법에는 다양한 방법들이 있지만, 먼저 간단하게 KFold 메서드 내에서 매개변수 **suffle**을 **True**로 설정해줄 수 있습니다.

데이터셋을 섞어주는 것이죠.

In [43]:
kfold = KFold(n_splits=3, shuffle=True)
n_iter =0
for train_index, test_index in kfold.split(feature):  
    x_train, x_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)


#1 교차 검증 정확도 : 0.92,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   3   4   9  11  14  17  20  22  23  24  26  27  31  35  38  39  45
  47  50  56  57  62  65  67  75  76  79  80  81  84  90  91  94  98 102
 103 104 105 107 108 112 113 119 129 135 136 138 144 147]

#2 교차 검증 정확도 : 0.94,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [  2   6   7   8  15  18  30  32  33  43  46  51  53  55  59  61  64  66
  69  70  74  77  78  82  83  85  89  93  97  99 106 109 110 117 120 121
 122 123 124 125 126 128 130 132 134 137 139 140 148 149]

#3 교차 검증 정확도 : 0.98,  학습 데이터 크기 : 100,  검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [  1   5  10  12  13  16  19  21  25  28  29  34  36  37  40  41  42  44
  48  49  52  54  58  60  63  68  71  72  73  86  87  88  92  95  96 100
 101 111 114 115 116 118 127 131 133 141 142 143 145 146]


 shuffle을 사용하여 데이터셋을 섞었기 때문에 index를 보면 순서대로가 아닌 임의로 데이터가 분배되었음을 확인할 수 있습니다.

다행히도 아까와 달리 모델의 학습과 검증이 나름 잘 일어난 것을 볼 수 있습니다.

### **5. Stratified K-Fold Cross Validation(계층별 k-겹 교차 검증)**

이러한 문제점을 해결하기 위한 또 다른 방법이 바로 **Stratified K-Fold Cross Validation(계층별 k-겹 교차 검증)**인데요.

계층별 k-겹 교차 검증은 원본 데이터의 전체 **레이블 분포**를 학습 및 검증 데이터셋에 반영해줍니다.

가령 iris data set 에서 레이블 0,1,2가 1:1:1 의 비율로 데이터가 분포되어있다면 학습데이터와 검증데이터의 레이블 분포도 0,1,2가 각각 1:1:1을 모두 따르게 되는 것이죠.

따라서 학습 데이터셋과 검증 데이터셋은 레이블 분포가 유사하도록 데이터셋을 분배하기 때문에, 데이터의 편향으로 인한 문제점을 방지할 수 있습니다.

코드를 통해 계층별 K겹 교차 검증을 구현해볼텐데요, 사용법은 KFold와 동일합니다!

먼저, **StratifiedKFold** 메서드를 import 해주었습니다.

In [45]:
from sklearn.model_selection import StratifiedKFold

S_kfold라는 계층별 5겹 교차검증 객체를 생성해주었습니다.

In [46]:
S_kfold = StratifiedKFold(n_splits=5)

KFold와 동일하게, 폴드별로 정확도와 인덱스를 확인해보겠습니다.

In [47]:
cv_accuracy = []
n_iter =0

for train_index, test_index in S_kfold.split(feature, label):  
    x_train, x_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    n_iter += 1
    
    accuracy = np.round(accuracy_score(y_test, pred), 4) 
    train_size = x_train.shape[0]
    test_size = x_test.shape[0]
    
    print('\n#{0} 교차 검증 정확도 : {1},  학습 데이터 크기 : {2},  검증 데이터 크기 : {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스 : {1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 0.9667,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109]

#2 교차 검증 정확도 : 0.9667,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#2 검증 세트 인덱스 : [ 10  11  12  13  14  15  16  17  18  19  60  61  62  63  64  65  66  67
  68  69 110 111 112 113 114 115 116 117 118 119]

#3 교차 검증 정확도 : 0.9,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#3 검증 세트 인덱스 : [ 20  21  22  23  24  25  26  27  28  29  70  71  72  73  74  75  76  77
  78  79 120 121 122 123 124 125 126 127 128 129]

#4 교차 검증 정확도 : 1.0,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#4 검증 세트 인덱스 : [ 30  31  32  33  34  35  36  37  38  39  80  81  82  83  84  85  86  87
  88  89 130 131 132 133 134 135 136 137 138 139]

#5 교차 검증 정확도 : 1.0,  학습 데이터 크기 : 120,  검증 데이터 크기 : 30
#5 검증 세트 인덱스 : [ 40  41  42  43  44  45  46  47  48  49  90  91  92  93  94  95  96  97
  98  99 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.96668


평균 검증 정확도가 눈에 띄게 상승했습니다!

그럼 레이블 분포가 학습 데이터셋과 검증 데이터셋에 어떻게 분포되어있는지 확인해보겠습니다.

In [50]:
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

전체 데이터셋은 레이블 0, 1, 2가 각각 1:1:1로 분포되어 있네요.

In [49]:
skfold_1 = StratifiedKFold(n_splits=5)
n_iter_=0

for train_index, test_index in skfold_1.split(iris_df, iris_df['label']):  
    n_iter_+=1
    train = iris_df['label'].iloc[train_index]
    test = iris_df['label'].iloc[test_index]
    
    print('# 교차 검증:{}'.format(n_iter_))
    print('# 학습 레이블 데이터 : \n',train.value_counts())
    print('# 검증 레이블 데이터 : \n',test.value_counts())

# 교차 검증:1
# 학습 레이블 데이터 : 
 0    40
1    40
2    40
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    10
1    10
2    10
Name: label, dtype: int64
# 교차 검증:2
# 학습 레이블 데이터 : 
 0    40
1    40
2    40
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    10
1    10
2    10
Name: label, dtype: int64
# 교차 검증:3
# 학습 레이블 데이터 : 
 0    40
1    40
2    40
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    10
1    10
2    10
Name: label, dtype: int64
# 교차 검증:4
# 학습 레이블 데이터 : 
 0    40
1    40
2    40
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    10
1    10
2    10
Name: label, dtype: int64
# 교차 검증:5
# 학습 레이블 데이터 : 
 0    40
1    40
2    40
Name: label, dtype: int64
# 검증 레이블 데이터 : 
 0    10
1    10
2    10
Name: label, dtype: int64


결과를 보니, 모든 학습 데이터셋과 검증 데이터셋에서 0, 1, 2 각각의 레이블이 1:1:1로 분포하고 있습니다!

이렇게 이번 게시물에서는 KFold 와 Stratified K-Fold 교차검증법을 구현해보았습니다.

정확하고 성능 좋은 모델을 위해서는 교차검증법이 필수적인 것 같아요!

다음 포스팅에서는 다른 교차검증법들을 실습해보겠습니다!

읽어주셔서 감사합니다:)